In [3]:
from collections import Counter, defaultdict
from functools import partial
import math, random

def entropy(class_probabilities):
    return sum(-p * math.log(p, 2) 
               for p in class_probabilities 
               if p)

def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

def partition_entropy(subsets):
    total_count = sum(len(subset) for subset in subsets)
    
    return sum( data_entropy(subset) * len(subset) / total_count
                for subset in subsets )

def partition_by(inputs, attribute):
    groups = defaultdict(list)
    for input in inputs:
        key = input[0][attribute]
        groups[key].append(input)
    return groups

def partition_entropy_by(inputs,attribute):
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

def classify(tree, input):
    if tree in [True, False]:
        return tree

    attribute, subtree_dict = tree

    subtree_key = input.get(attribute)  

    if subtree_key not in subtree_dict: 
        subtree_key = None              

    subtree = subtree_dict[subtree_key] 
    return classify(subtree, input)     

def build_tree_id3(inputs, split_candidates=None):
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()

    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues

    if num_trues == 0:                  
        return False                    

    if num_falses == 0:                 
        return True                     

    if not split_candidates:            
        return num_trues >= num_falses  

    best_attribute = min(split_candidates,
        key=partial(partition_entropy_by, inputs))

    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                      if a != best_attribute]

    subtrees = { attribute : build_tree_id3(subset, new_candidates)
                 for attribute, subset in partitions.items() }

    subtrees[None] = num_trues > num_falses

    return (best_attribute, subtrees)

In [4]:
data_20 = [
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'yes'},False),
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'no'},False),
        ({'level':'Junior','lang':'C#','tweets':'no','phd':'no'},True),
        ({'level':'Junior','lang':'C#','tweets':'no','phd':'no'},True),
        ({'level':'Senior','lang':'R','tweets':'no','phd':'yes'},True),
        ({'level':'Mid','lang':'C#','tweets':'yes','phd':'yes'},False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},False),
        ({'level':'Senior','lang':'C#','tweets':'yes','phd':'no'},False),
        ({'level':'Senior','lang':'R','tweets':'no','phd':'no'},True),
        ({'level':'Mid','lang':'C#','tweets':'no','phd':'no'},True),
        ({'level':'Mid','lang':'C#','tweets':'no','phd':'yes'},True),
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'yes'},True),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},False),
        ({'level':'Mid','lang':'R','tweets':'no','phd':'yes'},True),
        ({'level':'Senior','lang':'C#','tweets':'no','phd':'yes'},True),
        ({'level':'Senior','lang':'R','tweets':'no','phd':'yes'},True),
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'no'},True),
        ({'level':'Senior','lang':'C#','tweets':'yes','phd':'yes'},False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},False),
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'no'},False)
    ]

print("Drzewo decyzje - 20 kandydatów")
tree = build_tree_id3(data_20)
print(tree)

Drzewo decyzje - 20 kandydatów
('tweets', {'yes': ('level', {'Junior': ('phd', {'yes': ('lang', {'Java': True, None: False}), 'no': ('lang', {'Java': False, None: False}), None: False}), 'Mid': False, 'Senior': False, None: False}), 'no': ('lang', {'C#': True, 'R': True, 'Python': False, None: True}), None: True})


In [5]:
data_50 = [
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'yes'},False),
        ({'level':'Senior','lang':'Java','tweets':'yes','phd':'no'},False),
        ({'level':'Mid','lang':'C#','tweets':'no','phd':'no'},True),
        ({'level':'Junior','lang':'C#','tweets':'no','phd':'no'},True),
        ({'level':'Junior','lang':'R','tweets':'no','phd':'yes'},True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},False),
        ({'level':'Senior','lang':'C#','tweets':'yes','phd':'no'},False),
        ({'level':'Senior','lang':'R','tweets':'no','phd':'no'},True),
        ({'level':'Junior','lang':'C#','tweets':'no','phd':'no'},True),
        ({'level':'Senior','lang':'C#','tweets':'no','phd':'yes'},True),
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'yes'},True),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},False),
        ({'level':'Mid','lang':'R','tweets':'no','phd':'yes'},True),
        ({'level':'Senior','lang':'C#','tweets':'no','phd':'yes'},True),
        ({'level':'Junior','lang':'R','tweets':'no','phd':'yes'},True),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},True),
        ({'level':'Senior','lang':'C#','tweets':'yes','phd':'yes'},False),
        ({'level':'Mid','lang':'C#','tweets':'no','phd':'no'},False),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},True),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},True),
        ({'level':'Senior','lang':'C#','tweets':'no','phd':'yes'},True),
        ({'level':'Mid','lang':'C#','tweets':'no','phd':'no'},True),
        ({'level':'Junior','lang':'C#','tweets':'no','phd':'no'},True),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},True),
        ({'level':'Junior','lang':'C#','tweets':'yes','phd':'yes'},False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},True),
        ({'level':'Senior','lang':'C#','tweets':'yes','phd':'no'},False),
        ({'level':'Mid','lang':'R','tweets':'no','phd':'no'},True),
        ({'level':'Junior','lang':'C#','tweets':'no','phd':'no'},True),
        ({'level':'Mid','lang':'C#','tweets':'yes','phd':'yes'},True),
        ({'level':'Junior','lang':'R','tweets':'no','phd':'no'},True),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},False),
        ({'level':'Mid','lang':'R','tweets':'no','phd':'yes'},False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'no'},False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},False),
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'no'},False),
        ({'level':'Senior','lang':'Java','tweets':'yes','phd':'yes'},True),
        ({'level':'Mid','lang':'C#','tweets':'no','phd':'no'},False),
        ({'level':'Senior','lang':'Java','tweets':'yes','phd':'no'},False),
        ({'level':'Junior','lang':'Java','tweets':'no','phd':'no'},False),
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'yes'},False),
        ({'level':'Mid','lang':'C#','tweets':'no','phd':'no'},False),
        ({'level':'Junior','lang':'C#','tweets':'no','phd':'no'},True),
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'no'},True),
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'no'},True),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},True),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},True),
        ({'level':'Senior','lang':'R','tweets':'no','phd':'no'},True),
        ({'level':'Mid','lang':'Java','tweets':'no','phd':'no'},True),
]

print("Drzewo decyzje - 50 kandydatów")
tree = build_tree_id3(data_50)
print(tree)

Drzewo decyzje - 50 kandydatów
('tweets', {'yes': ('level', {'Junior': ('lang', {'Java': ('phd', {'yes': False, 'no': True, None: False}), 'R': False, 'C#': False, None: False}), 'Senior': ('lang', {'Java': ('phd', {'no': False, 'yes': True, None: False}), 'C#': False, 'R': ('phd', {'no': False, None: False}), None: False}), 'Mid': ('lang', {'R': ('phd', {'yes': True, 'no': False, None: False}), 'Java': True, 'C#': True, None: True}), None: False}), 'no': ('level', {'Mid': ('lang', {'C#': ('phd', {'no': False, None: False}), 'R': ('phd', {'yes': True, 'no': True, None: True}), 'Java': True, None: True}), 'Junior': ('lang', {'C#': True, 'R': True, 'Java': False, None: True}), 'Senior': True, None: True}), None: True})
